# dependencias

In [1]:
import igraph as ig
import json
import numpy as np
import sys
import os 
if '../src' not in sys.path:
    sys.path.append('../src')
import utilidadest5 as ut

# lectura de los datos 

In [2]:
# leemos el json
# Estadística,Biología, Farmacia, Física, Geociencias, Matemáticas y Química.
facultades = ['Estadistica', 'Biologia', 'Farmacia', 'Fisica', 'Geociencias', 'Matematicas', 'Quimica']
with open('../input/graph/Universidad Nacional de Colombia-Bogotá-FACULTAD DE CIENCIAS-Departamento de Estadística.json') as f:
    data_est = json.load(f)
with open('../input/graph/Universidad Nacional de Colombia-Bogotá-FACULTAD DE CIENCIAS-Departamento de Biología.json') as f:
    data_bio = json.load(f)
with open('../input/graph/Universidad Nacional de Colombia-Bogotá-FACULTAD DE CIENCIAS-Departamento de Farmacia.json') as f:
    data_far = json.load(f)
with open('../input/graph/Universidad Nacional de Colombia-Bogotá-FACULTAD DE CIENCIAS-Departamento de Física.json') as f:
    data_fis = json.load(f)
with open('../input/graph/Universidad Nacional de Colombia-Bogotá-FACULTAD DE CIENCIAS-Departamento de Geociencias.json') as f:
    data_geo = json.load(f)
with open('../input/graph/Universidad Nacional de Colombia-Bogotá-FACULTAD DE CIENCIAS-Departamento de Matemáticas.json') as f:
    data_mat = json.load(f)
with open('../input/graph/Universidad Nacional de Colombia-Bogotá-FACULTAD DE CIENCIAS-Departamento de Química.json') as f:
    data_qui = json.load(f)    

data = [data_est, data_bio, data_far, data_fis, data_geo, data_mat, data_qui]

In [3]:
# nodos 
nodos = [nodos['nodes'] for nodos in data]
# aristas
aristas = [aristas['links'] for aristas in data]
# info aparte 
options = [options['options'] for options in data]

In [4]:
id_evitar = [nodo[0]['id'] for nodo in nodos]
id_evitar

['2d4d5a17-7ef0-4fc4-8b25-d1af93445f50',
 'e0a9295b-404c-4134-99f0-756e8849413a',
 'a43e06fd-60e5-48c6-8a38-ed271b978ea4',
 '425bda01-002d-4b05-95fb-71196da8736e',
 'f9f44c87-afce-4bd3-a5aa-4988976fa40b',
 '9b94b0ad-bff6-4f20-ae80-170d728a6b9c',
 'c290f8c7-1919-44c2-8e05-111249363db7']

In [5]:
# Estadística,Biología, Farmacia, Física, Geociencias, Matemáticas y Química.
G_est= ut.grafo_biparito_t5(nodos[0], aristas[0],id_evitar[0])
G_bio= ut.grafo_biparito_t5(nodos[1], aristas[1],id_evitar[1])
G_far= ut.grafo_biparito_t5(nodos[2], aristas[2],id_evitar[2])
G_fis= ut.grafo_biparito_t5(nodos[3], aristas[3],id_evitar[3])
G_geo= ut.grafo_biparito_t5(nodos[4], aristas[4],id_evitar[4])
G_mat= ut.grafo_biparito_t5(nodos[5], aristas[5],id_evitar[5])
G_qui= ut.grafo_biparito_t5(nodos[6], aristas[6],id_evitar[6])
Grafos = [G_est, G_bio, G_far, G_fis, G_geo, G_mat, G_qui]

In [6]:
adyacencia = [] 
autores_list = []
for G in Grafos:
    # matriz de adyacencia
    A = np.array(G.get_biadjacency()[0]) 
    autores = G.vs.select(type=False)['label']
    autores_list.append(autores)
    print(len(autores))
    # A * A^t
    A_A_t = np.dot(A, A.T)
    # remplazamos la diagonal con ceros 
    np.fill_diagonal(A_A_t, 0)
    adyacencia.append(A_A_t)
    

58
108
90
146
82
192
210


In [7]:
grafos_autores = []
for i in range(len(adyacencia)):
    grafos_autores.append(ut.grafo_matriz_adyacencia_t5(adyacencia[i], autores_list[i]))

In [8]:
for grafo_autores in grafos_autores:
    print(grafo_autores.summary())

IGRAPH U-W- 58 706 -- 
+ attr: label (v), weight (e)
IGRAPH U-W- 108 1584 -- 
+ attr: label (v), weight (e)
IGRAPH U-W- 90 1031 -- 
+ attr: label (v), weight (e)
IGRAPH U-W- 146 2607 -- 
+ attr: label (v), weight (e)
IGRAPH U-W- 82 510 -- 
+ attr: label (v), weight (e)
IGRAPH U-W- 192 2148 -- 
+ attr: label (v), weight (e)
IGRAPH U-W- 210 6237 -- 
+ attr: label (v), weight (e)


In [10]:
for i in range(len(grafos_autores)):
    layout = grafos_autores[i].layout("kk")
    layout.scale(1.0)
    # tranparencia de los nodos
    grafos_autores[i].vs['color'] =  "#00BFFF80"
    
    ig.plot(grafos_autores[i], layout = layout, bbox = (1000, 1000), margin = 20,vertex_label= None,vertex_size=10 ).save('../output/taller 5_plot/grafo_autores_'+ facultades[i] +'.png')


In [ ]:
# subgrafo conexo
G_autores_conexo = grafos_autores[0].connected_components().giant()
ig.plot(G_autores_conexo, layout = layout, bbox = (1000, 1000), margin = 20, vertex_size = 10, vertex_label_dist = 10).save('../output/taller 5_plot/grafo_autores_conexo.png')


In [ ]:
sum_pesos_aristas = G_autores_conexo.strength(weights='weight')

# Normalizar la suma de los pesos de las aristas para ajustarla al rango de tamaños de los nodos
max_sum_pesos_aristas = max(sum_pesos_aristas)
sizes = [x / max_sum_pesos_aristas * 40 for x in sum_pesos_aristas]  # Ajusta el rango de tamaño deseado
G_autores_conexo.vs["size"] = sizes

In [ ]:
dendrogram = G_autores_conexo.community_edge_betweenness(weights="weight")
layout_autores_conexo = G_autores_conexo.layout("kk")

# Obtener las comunidades finales
communities = dendrogram.as_clustering()

# Obtener el número de comunidades
num_communities = len(communities)

# Asignar colores a cada comunidad
palette = ig.RainbowPalette(n=num_communities)
community_colors = [palette.get(i) for i in communities.membership]


# Definir transparencia para los nodos
node_transparency = 0.5  # Valor entre 0 y 1, donde 0 es completamente transparente y 1 es completamente opaco

# Convertir los colores de las comunidades a RGBA con la transparencia deseada
community_colors_with_alpha = [color[:-1] + (node_transparency,) for color in community_colors]

# Dibujar el grafo con nodos semi-transparentes
ig.plot(G_autores_conexo,'../output/taller4_plot/caso_6/comunidades_6.png', layout=layout_autores_conexo,vertex_color=community_colors_with_alpha,vertex_label= None,).save('../output/taller 5_plot/comunidades_autores_conexo.png')